## Adaptive MD

Using __`radical.pilot`__

For the NTL9 system

In [1]:
import os
import sys

In [2]:
# set default verbose level
verbose = os.environ.get('RADICAL_PILOT_VERBOSE', 'REPORT')
os.environ['RADICAL_PILOT_VERBOSE'] = verbose

# set default URL to IMP Mongo DB
# path_to_db = os.environ.get(
#     'RADICAL_PILOT_DBURL', "mongodb://ensembletk.imp.fu-berlin.de:27017/rp")

# assume we run a local
path_to_db = os.environ.get(
    'RADICAL_PILOT_DBURL', "mongodb://localhost:27017/rp")

os.environ['RADICAL_PILOT_DBURL'] = path_to_db

In [3]:
import radical.pilot as rp
import radical.utils as ru

/Users/jan-hendrikprinz/anaconda/lib/python2.7/site-packages/radical/utils/atfork/stdlib_fixer.py:58: UserWarning: logging module already imported before fixup.
  warnings.warn('logging module already imported before fixup.')


In [5]:
# import adaptive components

from adaptivemd import OpenMMEngine, AllegroCluster, Brain, MDCluster, \
    LocalCluster, File, NTrajectories
import adaptivemd.util as amp

#### 1. Select a resource

We pick to run on this machine

In [6]:
resource = LocalCluster(runtime=5, cores=1)

We still need to add the necessary path to find the conda installation

In [7]:
resource.add_path(amp.path_conda_local_jhp)

In [8]:
# we use a reporter class for nicer output
report = ru.LogReporter(name='radical.pilot', level=verbose)
report.title('Getting Started (RP version %s)' % rp.version)


 Getting Started (RP version v0.44.RC1)                                         



#### 2. Create the engine

pick a file from the currend working directory

In [9]:
pdb_file = File('file://files/input.pdb')

In [10]:
engine = OpenMMEngine(
    pdb_file=pdb_file,
    system_file=File('file://files/system.xml'),
    integrator_file=File('file://files/integrator.xml')
)

#### 3. Create the cluster used

In [12]:
cluster = MDCluster(
    system='ntl9',
    resource=resource,
    report=report)

In [11]:
# add the path to CONDA if now already in the default
cluster.add_path(os.environ.get('~/anaconda/bin'))

select a strategy aka _brain_

In [12]:
brain = Brain(cluster)  # this needs to be smarter

So let's do it...

In [13]:
with cluster:
    report.ok('>>ok\n')

    report.header('submit units')

    trajs = [
        cluster.get_new_trajectory_file(pdb_file, 2)
        for x in range(2)
    ]

    # submit the trajectories
    cluster.submit_trajectories(trajs)

    # Run immediately 4 more trajectories
    event = DoMoreTrajectories(
        when=NTrajectories(cluster, 1),
        unless=NTrajectories(cluster, 8),
        number=4,
        length=2
    )
    cluster.add_event(event)
    cluster.wait()

report.header('generated new trajectories')

for f in cluster.files:
    report.info(repr(f) + '\n\t')

report.header()

new session: [rp.session.Stevie.fritz.box.jan-hendrikprinz.017177.0010]        \
database   : [mongodb://localhost:27017/rp]                                   ok
create pilot manager                                                          ok
create unit manager                                                           ok

--------------------------------------------------------------------------------
submit pilots                                                                   

create pilot description [local.localhost:1]                                  ok
submit 1 pilot(s) .                                                           ok
add 1 pilot(s)                                                                ok
stage shared data from engine                                                 ok

--------------------------------------------------------------------------------
submit units                                                                    

submit 2 unit(s)
       

In [16]:
len(cluster.files)

8